# Example to add thermal noise to simulations
This notebook provides an example of how to add thermal noise as different components.\
The identified components are:
* Ice
* Galaxy
* Electronics
  
Each of these components uses a different module from NuRadio

Note that this illustrates the procedure to simulate thermal noise used in the calibration of the absolute system response of the RNO-G experiment.

This thermal noise is constructed from first principle, for data-driven noise one should use a different procedure e.g. parameterized Rayleigh noise

In [ ]:
import datetime
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import constants

import NuRadioReco
from NuRadioReco.detector.RNO_G.rnog_detector import Detector
from NuRadioReco.framework.event import Event
from NuRadioReco.framework.station import Station
from NuRadioReco.framework.channel import Channel
from NuRadioReco.utilities import units

In [ ]:
logger = logging.getLogger("NuRadioReco")
logger.setLevel(logging.CRITICAL)

We start by generating a set of events with the same structure as simulated events. We do this to use the module.run() convention from NuRadio.

In [ ]:
example_station = 11
channel_id=0

sampling_rate = 3.2 * units.GHz
nr_samples = 2048
frequencies = np.fft.rfftfreq(nr_samples, d=1./sampling_rate)

nr_events = 10

In [ ]:
def generate_event():
    event = Event(run_number=-1, event_id=-1)
    station = Station(example_station)
    station.set_station_time("2023-08-01", format="isot")
    channel_ids = [0]
    for channel_id in channel_ids:
        channel = Channel(channel_id)
        channel.set_frequency_spectrum(np.zeros_like(frequencies, dtype=np.complex128), sampling_rate)
        station.add_channel(channel)
    event.set_station(station)
    return event

## Components

We start of this example by showing how to generate each component individually. To see an example in which all three are immediately generated as one would in application, scroll to the end of the notebook.

## Ice noise

The thermal noise generated by the surrounding ice volume is calculated using the channelIceThermalNoiseAdder module. This module uses pre-generated effective temperature profiles. To create these one can use the code in NuRadioMC/examples/generate/simulate_effective_ice_temperature.py

The code propagates rays in the ice starting from the antenna at several incident angles. For each angle, the ice temperature along the ray path is integrated weighed by the local attenuation effects. This yields an effective temperature as seen by the antenna at a specific incident angle.

These profiles are folded into the antenna's vector effective lengths to yield a voltage. Hence the module yields thermal noise from ice <b>before</b> the amplifiers. To obtain the noise at readout one should fold in the system response using the HardwareResponseIncorporator module.

In [ ]:
eff_temp_dir = os.path.join(os.path.dirname(NuRadioReco.__file__), "examples", "RNOG", "thermal_noise", "eff_temperatures")
eff_temp_files = [os.path.join(eff_temp_dir, eff_temp_filename) for eff_temp_filename in ["eff_temperature_-1.0m_ntheta100_GL3.json",
                                                                                          "eff_temperature_-40m_ntheta100.json",
                                                                                          "eff_temperature_-100m_ntheta100_GL3.json"]]

In [ ]:
from NuRadioReco.modules.channelIceThermalNoiseAdder import channelIceThermalNoiseAdder

detector = Detector(database_connection="RNOG_public", log_level=logging.NOTSET,
                   select_stations=11)
detector_time = datetime.datetime(2023, 8, 1)
detector.update(detector_time)

ice_events = [generate_event() for e in range(nr_events)]
ice_noise_adder = channelIceThermalNoiseAdder()
ice_noise_adder.begin(eff_temperature_files=eff_temp_files)
for event in ice_events:
    station = event.get_station()
    ice_noise_adder.run(event, station, detector=detector)

In [ ]:
ice_spectra = []
for event in ice_events:
    station = event.get_station()
    channel = station.get_channel(channel_id)
    spectrum = channel.get_frequency_spectrum()
    ice_spectra.append(np.abs(spectrum))
fig, ax = plt.subplots()
ax.plot(frequencies, np.mean(ice_spectra, axis=0))
ax.grid()
ax.set_xlabel("freq / GHz")
ax.set_ylabel("spectral amplitude / V/GHz");

## Electronic noise

In principle the electronic noise should be frequency dependent and is ideally derived from a noise temperature measurement. Since these are not available, the electronic noise is approximated as flat with a certain noise temperature. From old measurements (see noise measurements in https://arxiv.org/abs/2411.12922 Figure 8) we approximate this temperature as 80 K.

In the context of the RNO-G system absolute amplitude calibration, an extra free parameter is added to the electronic noise profile to account for this uncertainty. This free parameter module is not included in this example.

In [ ]:
from NuRadioReco.utilities.signal_processing import calculate_vrms_from_temperature

electronic_noise_temp = 80 * units.K
min_freq = 10 * units.MHz
max_freq = 1500 * units.MHz
impedance = 50 * units.ohm

amplitude = calculate_vrms_from_temperature(electronic_noise_temp, [min_freq, max_freq], impedance=impedance)

In [ ]:
from NuRadioReco.modules.channelGenericNoiseAdder import channelGenericNoiseAdder

electronic_events = [generate_event() for e in range(nr_events)]
electronic_noise_adder = channelGenericNoiseAdder()
electronic_noise_adder.begin()

for event in electronic_events:
    station=event.get_station()
    electronic_noise_adder.run(event, station, detector,
                               amplitude=amplitude,
                               min_freq=min_freq, max_freq=max_freq,
                               type="rayleigh")

In [ ]:
electronic_spectra = []
for event in electronic_events:
    station = event.get_station()
    channel = station.get_channel(channel_id)
    spectrum = channel.get_frequency_spectrum()
    electronic_spectra.append(np.abs(spectrum))
fig, ax = plt.subplots()
ax.plot(frequencies, np.mean(electronic_spectra, axis=0))
ax.grid()
ax.set_xlabel("freq / GHz")
ax.set_ylabel("spectral amplitude / V/GHz");

## Galactic noise

Galactic noise uses the channelGalacticNoiseAdder module. This module reads noise temperatures from a preproduced skymap of the radio galaxy. These signals are propagated through air and ice to the desired antenna location, where the module folds this signal into the antenna vector effectve length.

In [ ]:
from NuRadioReco.modules.channelGalacticNoiseAdder import channelGalacticNoiseAdder

galactic_events = [generate_event() for e in range(nr_events)]
galactic_noise_adder = channelGalacticNoiseAdder()
galactic_noise_adder.begin(freq_range=[min_freq, max_freq],
                           caching=True)

for event in galactic_events:
    station = event.get_station()
    galactic_noise_adder.run(event, station, detector)

In [ ]:
galactic_spectra = []
for event in galactic_events:
    station = event.get_station()
    channel = station.get_channel(channel_id)
    spectrum = channel.get_frequency_spectrum()
    galactic_spectra.append(np.abs(spectrum))
fig, ax = plt.subplots()
ax.plot(frequencies, np.mean(galactic_spectra, axis=0))
ax.grid()
ax.set_xlabel("freq / GHz")
ax.set_ylabel("spectral amplitude / V/GHz");

### Total noise

In [ ]:
class threeComponentAdder():
   def __init__(self):
      self.ice_noise_adder = channelIceThermalNoiseAdder()
      self.electronic_noise_adder = channelGenericNoiseAdder()
      self.galactic_noise_adder = channelGalacticNoiseAdder()

   def begin(self, freq_range, eff_temp_files, electronic_noise_temp, impedance):
      self.freq_range = freq_range
      self.electronic_noise_amplitude = calculate_vrms_from_temperature(temperature=electronic_noise_temp,
                                                                        bandwidth=[freq_range[0], freq_range[1]],
                                                                        impedance=impedance)

      self.ice_noise_adder.begin(eff_temp_files)
      self.electronic_noise_adder.begin()
      self.galactic_noise_adder.begin(freq_range=freq_range,
                                      caching=True)
      return

   def run(self, event, station, detector, electronic_type):
      self.ice_noise_adder.run(event, station, detector)
      self.electronic_noise_adder.run(event, station, detector,
                                      amplitude=self.electronic_noise_amplitude,
                                      min_freq = self.freq_range[0],
                                      max_freq = self.freq_range[1],
                                      type = electronic_type)
      self.galactic_noise_adder.run(event, station, detector)

In [ ]:
noise_events = [generate_event() for e in range(nr_events)]

three_component_adder = threeComponentAdder()
three_component_adder.begin(freq_range=[min_freq, max_freq],
                            eff_temp_files=eff_temp_files,
                            electronic_noise_temp=electronic_noise_temp,
                            impedance=impedance)

for event in noise_events:
    station = event.get_station()
    three_component_adder.run(event, station, detector, electronic_type="rayleigh")

In [ ]:
noise_spectra = []
for event in noise_events:
    station = event.get_station()
    channel = station.get_channel(channel_id)
    spectrum = channel.get_frequency_spectrum()
    noise_spectra.append(np.abs(spectrum))

### Plotting results

In [ ]:
alpha=0.7
fig, ax = plt.subplots()
ax.plot(frequencies, np.mean(ice_spectra, axis=0), label="Ice",
        lw=1., alpha=alpha)
ax.plot(frequencies, np.mean(electronic_spectra, axis=0), label="Electronic",
        lw=1., alpha=alpha)
ax.plot(frequencies, np.mean(galactic_spectra, axis=0), label="Galactic",
        lw=1., alpha=alpha)
ax.plot(frequencies, np.mean(noise_spectra, axis=0), label="Three components", lw=1.)
ax.set_xlabel("freq / GHz")
ax.set_ylabel("Spectral amplitude / V/GHz")
ax.legend()
ax.grid(alpha=0.3)
ax.set_title(f"Channel {channel_id}");